In [1]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import datetime

def collect_airline_names():
    url = 'https://www.airlinequality.com/review-pages/latest-airline-reviews/'

    html_text = requests.get(url).content
    soup = BeautifulSoup(html_text,"lxml")

    box = soup.find('ul',attrs={'class':'item'})
    box = box.find_all('li')
    list_of_airlines = []
    for l in box:
        list_of_airlines.append( l.get_text()) 

    return list_of_airlines

In [2]:


url = 'https://www.airlinequality.com/airline-reviews/aer-lingus/'
html_text = requests.get(url).content

list_of_reviews = []
soup = BeautifulSoup(html_text,'lxml')
article = soup.find('article',attrs={'itemprop' : 'review'})
date = str(datetime.date.today())
title = article.find('h2',attrs={'class':'text_header'}).get_text()
name = article.find('span',attrs={'itemprop':'name'}).get_text()
review_date = article.find('time',attrs={'itemprop' : 'datePublished'}).get_text()
overall_rating = article.find('span',attrs={'itemprop':'ratingValue'}).get_text()
review = article.find('div',attrs={'class':'text_content'}).get_text()

dictionary_of_reviews = {}
dictionary_of_reviews['date'] = date
dictionary_of_reviews['name'] = name 
dictionary_of_reviews['review_date'] = review_date
dictionary_of_reviews['title'] =title
dictionary_of_reviews['overall_rating'] = overall_rating
dictionary_of_reviews['review'] =review


list_of_reviews.append(dictionary_of_reviews)
list_of_reviews

[{'date': '2023-11-04',
  'name': 'H Robinson',
  'review_date': '30th October 2023',
  'title': '“The airline is absolutely terrible”',
  'overall_rating': '1',
  'review': "✅ Trip Verified | The airline is absolutely terrible. Our flight was canceled because the engine was broken and we were given no other option than to fly into a different city and take busses. Everyone was rude and we were treated very poorly. The planes were disgusting, the food wasn't edible, the seats were tiny, the planes are clearly not well maintained and everything was broken. I know the flights are cheap, but you may be taking a bus and traveling overnight unexpectedly. I suggest paying a little more to not have to be treated like a ticket instead of a person."}]

In [11]:
list_of_reviews = []
i = 1
while(True):
    url = f'https://www.airlinequality.com/airline-reviews/air-austral/page/{i}'
    html_text = requests.get(url).content

  
    soup = BeautifulSoup(html_text,'lxml')
    articles = soup.find_all('article',attrs={'itemprop' : 'review'})  #['ar1','ar2']
    if(articles==[]):
        break
    for article in articles:
        date = str(datetime.date.today())   
        title = article.find('h2',attrs={'class':'text_header'}).get_text()
        name = article.find('span',attrs={'itemprop':'name'}).get_text()
        review_date = article.find('time',attrs={'itemprop' : 'datePublished'}).get_text()
        overall_rating = article.find('span',attrs={'itemprop':'ratingValue'})
        if(overall_rating==None):
            overall_rating=0
        else:
            overall_rating = overall_rating.get_text()
        review = article.find('div',attrs={'class':'text_content'}).get_text()

        d = {}

        table = article.find('table',attrs={'class':'review-ratings'})

        table_rows = table.find_all('tr')   

        for table_row in table_rows:
            table_datas = table_row.find_all('td')   #[t , b]
            key = table_datas[0].get_text()
            if('stars' in table_datas[1]['class']):
                value = len(table_datas[1].find_all('span',attrs={'class':'star fill'}))   
            else:
                value = table_datas[1].get_text()
        
            d[key] = value
            
        



        dictionary_of_reviews = {}
        dictionary_of_reviews['airline'] = url.split('/')[-3]
        dictionary_of_reviews['date'] = date
        dictionary_of_reviews['name'] = name 
        dictionary_of_reviews['review_date'] = review_date
        dictionary_of_reviews['title'] =title
        dictionary_of_reviews['overall_rating'] = overall_rating
        dictionary_of_reviews['review'] =review
        dictionary_of_reviews['detail']= d

    






    list_of_reviews.append(dictionary_of_reviews)

    print(f'Collected data from page {i}')
    i = i + 1
        


df = pd.json_normalize(list_of_reviews)
df

Collected data from page 1
Collected data from page 2
Collected data from page 3
Collected data from page 4
Collected data from page 5


,airline,date,name,review_date,title,overall_rating,review,detail.Type Of Traveller,detail.Seat Type,detail.Route,detail.Date Flown,detail.Seat Comfort,detail.Cabin Staff Service,detail.Ground Service,detail.Value For Money,detail.Recommended
0,air-austral,2023-11-04,Gowtham Thangavel,1st May 2019,"""really bad service""",1,✅ Trip Verified | Reunion to Chennai. We were...,Couple Leisure,First Class,Reunion to Chennai,April 2019,2.0,2.0,1.0,1.0,no
1,air-austral,2023-11-04,T G Vernet,22nd December 2014,Air Austral customer review,7,CDG-RUN return. Air Austral is the airline of ...,NaN,Economy Class,NaN,NaN,NaN,NaN,NaN,4.0,yes
2,air-austral,2023-11-04,M Louise,2nd August 2011,Air Austral customer review,8,SYD to Mauritius. Excellent service and quite ...,NaN,Economy Class,NaN,NaN,NaN,NaN,NaN,4.0,yes
3,air-austral,2023-11-04,G Luyten,17th May 2009,Air Austral customer review,4,Only flew Air Austral for medium haul (RUN-SEY...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no
4,air-austral,2023-11-04,Bernard Ative,24th January 2004,Air Austral customer review,0,I'm used to fly with the french national airli...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no
